<a href="https://colab.research.google.com/github/PyDataOsaka/handson_pytorch/blob/master/cnn_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch hands-on (CNN)

Adapted from [here](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)

In [0]:
!rm -r ./log

In [0]:
%tensorflow_version 2.x
%load_ext tensorboard

In [0]:
%tensorboard --logdir ./log

In [0]:
from time import time
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import torchvision
import torchvision.transforms as transforms

## Load image data

* [CIFAR10](https://pytorch.org/docs/stable/torchvision/datasets.html#cifar) dataset
* Compose data preprocesses
  * Convert from numpy to pytorch Tensor (`transforms.ToTensor()`)
  * Channel-wise normalization (`transforms.Normalize()`)
    * https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.Normalize
* Make `DataLoader` for automatic minibatching
  * https://pytorch.org/docs/stable/data.html

In [0]:
def get_data(batch_size: int=64):
  transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

  trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                          download=True, transform=transform)
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=2)

  testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                         download=True, transform=transform)
  testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                           shuffle=False, num_workers=2)

  classes = ('plane', 'car', 'bird', 'cat',
             'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
  
  return trainloader, testloader, classes

## Define CNN model

In [0]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool1 = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.pool2 = nn.MaxPool2d(2, 2)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool1(F.relu(self.conv1(x)))
    x = self.pool2(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [0]:
model = Net()
trainloader, _, _ = get_data()
writer = SummaryWriter("./log/1")
writer.add_graph(model, (trainloader.__iter__().__next__()[0]))
writer.close()

## Define functions

* Transfer arrays from cpu to gpu
  * `model.to(device)`, `data[0].to(device), data[1].to(device)`
  * https://pytorch.org/docs/stable/notes/cuda.html

### Training

In [0]:
def train(model: nn.Module, trainloader, log_dir: str, device="cpu"):
  model.to(device)

  loss = nn.CrossEntropyLoss()
  opt = optim.Adam(model.parameters(), lr=0.001)

  writer = SummaryWriter(log_dir)
  running_loss = 0.0
  prev_time = time()
  n_minibatches = 0

  for epoch in range(4):
    for i, data in enumerate(trainloader, 0):
      # get the inputs; data is a list of [inputs, labels]
      inputs = data[0].to(device)
      labels = data[1].to(device)

      # zero the parameter gradients
      opt.zero_grad()

      # forward + backward + optimize
      outputs = model(inputs)
      loss_value = loss(outputs, labels)
      loss_value.backward()
      opt.step()

      writer.add_scalar("loss_value", loss_value, n_minibatches)
      n_minibatches += 1

      # print statistics
      running_loss += loss_value.item()
      if i % 100 == 99:    # print every 100 mini-batches
        print('[{}, {:5d}] loss: {:.3f}, elapsed time: {:.1f} [sec]'.format(
              epoch + 1, i + 1, running_loss / 100, time() - prev_time))
        running_loss = 0.0
        prev_time = time()

### Prediction

In [0]:
def evaluate(model: nn.Module, testloader, device: str="cpu"):
  correct = 0
  total = 0

  with torch.no_grad():
    for data in testloader:
      inputs = data[0].to(device)
      labels = data[1].to(device)
      outputs = model(inputs)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))

## Training and evaluation of the model on CPU

In [0]:
trainloader, testloader, classes = get_data()
model = Net()

train(model, trainloader, "./log/1")
torch.save({
    "model": model.state_dict(),
}, "./model_cpu.pt")

evaluate(model, testloader)

## Training and evaluation of the model on GPU

In [0]:
trainloader, testloader, classes = get_data()
model = Net()

train(model, trainloader, "./log/2", "cuda")
torch.save({
    "model": model.state_dict(),
}, "./model_gpu.pt")

evaluate(model, testloader, "cuda")

## Load trained model

In [0]:
trainloader, testloader, classes = get_data()
model = Net()

checkpoint = torch.load("./model_gpu.pt")
model.load_state_dict(checkpoint["model"])
model.to("cuda")

evaluate(model, testloader, "cuda")